# WeRateDogs Twitter Account - Data Wrangling Report

## 1) Gather Data

The data used for the WeRateDogs Twitter Account Analysis Report was gathered from three different sources:

1) The 'twitter_archive_enhanced.csv' file, which was available for manual download.  This file contained information on the tweet ID's, text, dog rating, dog stage etc. This file was opened in a dataframe entitled 'twitter_archive'.

2) The 'image_predictions.tsv' file, which was downloaded programmaticaly, using the Requests library, from Udacity's server. This file contained information on the breed of dog (or other object, animal) that was predicted from the image in each tweet using neural networks.  This file was opened in a dataframe entitled 'image_predictions'.

3) The 'tweet_json.txt' file, which was obtained by quering the Twitter API, using the tweet IDs from the 'twitter_archive_enhanced.csv' file.  However, 11 errors were obtained during this process, with error code: 144. This error code indicates that no status was found with that ID number, and it corresponds with HTTP 404: The requested Tweet ID is not found (if it existed, it was probably deleted).  The code was run again in an attempt to gather the data for the 11 tweet_id's in the tweet_error list, however the same error messages were received.  Using the Tweepy library, JSON data for 2345 tweets was downloaded and stored in the 'tweet_json.txt' file.  This file contained information about each tweet's retweet count and favorite ("like") count, and was opened in a dataframe entitled 'df_json'.


## 2) Assess Data

The data in all three dataframes was assessed for quality and tidiness issues:

### Quality
#### Twitter_archive Table
-  2278 missing entries in 'in_reply_to_status_id', 'in_reply_to_user_id' columns
-  2175 missing entries in 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp' columns
-  59 missing entries in 'expanded urls' column
-  'timestamp' and 'retweeted timestamp' are objects but should be datetime datatypes
-  'rating_numerator' & r'ating_denominator' have values higher than expected (15 - 1776 and 0, 11 - 170, respectively).
-  'in_reply_to_status_id', 'in_reply_to_user_id' 'retweeted_status_id', 'retweeted_status_user_id', columns are all floats, but they should be integers.
-  dog names and dog stages may not have been extracted correclty from the text
-  In 'time' column, +0000 at the end of the column
-  Convert dataytpe of newly added column 'dog_stage' to categorical variable

#### Image_predictions Table
-  Column headings are confusing
-  281 Missing rows. Have 2075 entries in image_predictions, but 2356 entries in twitter_archive table.
-  'p1', 'p2, 'p3' column has some strange dog names: web-site, cardigan,cheeseburger etc.
-  prediction confidence values not listed as percentages

#### df_json Table
-  2345 entries. therefore, 11 less entries than number of entries in twitter_archive (due to deleted tweets).  


### Tidiness

-  'Unnamed:0' column in df_json table is not needed
-  in twitter_archive table, 'Timestamp' column has both date and time in each observation.
-  'doggo' 'puppo' 'pupper' and 'floofer' are all types of dog stages and so should be under the column 'dog_stage' 
-  Twitter_archive & df_json tables can be combined


## 3) Clean Data

A copy was made of each dataframe, and the copied dataframes were cleaned in the order of missing data, tidiness issues and quality issues.

### Missing data

**1) 'twitter_archive_clean' Table: Retweeted data**
There were 2175 missing entries in the 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp' columns. However, the data in these columns were related to retweets.  Since we were only interested in original tweets:
-  The 181 rows in the twitter_archive table that have entries in the columns related to retweets ('retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp') were removed.
-  The columns related to retweets ('retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp') were removed. 

**2) 'twitter_archive_clean' Table: Reply Data**
There were 2278 missing entries in the 'in_reply_to_status_id', 'in_reply_to_user_id' columns. According to the Tweet Data Dictionary (https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object), 'in_reply_to_status_id' column contains the integer representation of the original Tweet’s ID, and indicates that the represented Tweet in this row is a reply. Also, the 'in_reply_to_user_id' contains the integer representation of the original Tweet’s author ID (this will not necessarily always be the user directly mentioned in the Tweet). Therefore, the rows in the twitter_archive table that contain non-null values in the 'in_reply_to_status_id' and 'in_reply_to_user_id', show that these tweets are a reply to another tweet. Since we were only interested in original tweets: 
-  The rows in the twitter_archvie_clean dataframe that contained non-null values in the 'in_reply_to_status_id' and 'in_reply_to_user_id' columns were removed. 
-  The 'in_reply_to_status_id' and 'in_reply_to_user_id' columns from the twitter_archive_clean dataframe were removed. 

**3) 'twitter_archive_clean' Table: Expanded url Data**

The 'expanded_urls' column contained the web links to photo's of the dogs.  For this project, we were only interested in ratings with images.  There were 59 missing entries in the 'expand_url' column in the twitter_archive_clean table.  After removing the the 'retweets' and 'replies' data from the twitter_archive table in the previous data cleaning step, there were only 3 missing entries left in the 'expanded_urls' column. The tweet_id's of these entries were not found in the image_predictions table, and furthermore, analysis of the text of these 3 entries indicates that they were general comments, and not posts of pictures of dogs with the corresponding rating. 

-  The 3 remaining missing entries in the 'expanded_url' column were removed from the twitter_archive_clean_2 table.

**4) 'twitter_archive_clean' Table: Deleted Tweets**

There were 2345 entries in df_json, which is 11 less entries than the number of entries in the twitter_archive_clean table. This was due to the 11 tweets that could not be collected when quering the twitter API, and the error message indicated that these tweets had been deleted.  The 11 'deleted tweets' were to be removed from the twitter_archive_clean_2 table, to prepare for merging this table with the df_json table in the 'Tidiness' section.  However, it was found that these 11 tweets were no longer present in the twitter_archive_clean_2 table. Following analysis of the original 'twitter_archive' table (before cleaning processes), it was found that these 11 tweets were related to retweets and had therefore already been removed from the twitter_archive_clean_2 table.

**5) 'image_predictions_clean' Table: Missing rows**

The image_predictions_clean table had 281 less rows than the original twitter_archive table.  Following the previous cleaning processes on the twitter_archive_clean table, to remove data related to retweets, replies, missing 'expanded_urls', and deleted tweets (tweet_errors), the difference in number of entries between the 2 tables was investigated again. 

-  Due to the fact that we only wanted original ratings (no retweets) that have images for this project, the tweet_id's from the twitter_archive table, related to retweets and replies, were removed from the image_predictions_clean table

-  Once these entries had been removed from the table, it was found that the tweet_id's collected in the tweet_errors list (i.e. the deleted tweets) were also not present in the image_predictions_remove table.

-  Following these changes, the image_predictions_clean table had 1971 entries, which is still 123 entries less than the twitter_archive_clean_2 table (with a total of 2094 entries). However, in line with the 'tidy data' rules, these two tables were not intended to be merged as image predictions and tweet information are seen as two different types of observational units. Therefore, further investigations into the difference in entries between the two dataframes was not required for the purpose of this project.


### Tidiness

**1) 'df_json_clean' Table: 'Umnamed: 0' column**

-  The 'Unnamed: 0' column from df_json_clean table was removed unsing the .drop() method.


**2) 'twitter_archive_table': Separate time & date data**

- The date and time data in the 'timestamp' column were separated into 2 different columns, using panda's regular expressions and the string.extract method

**3) 'twitter_archive_clean_2': Combine data in 'doggo', 'pupper', 'puppo', 'floofer' columns into 1 column called 'dog_stage'**

-  A subset of the twitter_archive_clean_2 dataframe, called 'dog', was created, with 'tweet_id', 'doggo' 'puppo' 'pupper' and 'floofer' columns. 
-  All 'None' entries were converted to NaN values.  
-  The ".combine_first" method was used to merge together the values from  'doggo' 'puppo' 'pupper' and 'floofer' columns into new 'dog stage' column. 
-  The 'dog_stage'column was merged into the twitter_archvie_clean_2 dataframe, and the 'doggo' 'puppo' 'pupper' and 'floofer' columns were removed from the dataframe using the .drop() method.

**4) Merge 'twitter_archive_clean_2' and 'df_json' Tables**

-  The 'twitter_archive_clean_2' and 'df_json' tables were merged together, joining on tweet_id, to create the 'twitter_archive_master' dataframe.

-  Note that, the image_predcitions table was not be merged with the twitter_archvie_master dataframe.  Considering the 'tidy data' rules, the two tables related to image predictions and tweet information can be seen as two different types of observational units.


### Quality

Note: The 'retweeted timestamp' 'in_reply_to_status_id', 'in_reply_to_user_id' 'retweeted_status_id' and 'retweeted_status_user_id', columns have been removed from the twitter_archive_master dataframe, therefore this datatype quality issue is no longer relevant.

**1) 'twitter_archive_master" Table: Datatype Conversion & Removing Text from 'time' Column**

-  The '+0000' string at the end of each value in the 'time' column was removed
-  The 'date' column was converted from object to datetime datatype. 
-  The 'dog_stage' was converted from object to categorical datatype.

**2) 'twitter_archive_master" Table: Rating Numerator & Denominator Entries**

There were 22 entries in the twitter_archive_master dataframe, whereby the 'rating_numerator' was higher and lower than expected (< 10 and > 15) and the 'rating_denomintar' was not equal to 10, as expected.  

- The 'text' column,from which these values were extracted, was checked for mistakes during the data extraction process. 
-  In the case that the data from the 'text' column was not extracted properly, change the rating_numerator and rating_denominator entries to their real values.  

**3) 'twitter_archive_master" Table: Missing Entries in the 'dog_stage' Column**

-  A subset of the twitter_archive_master dataframe was created for all of the rows with null values in the 'dog_stage' column. 
-  The 'text' column was investigated for words containing 'doggo', 'floofer', 'puppo', 'pupper' using the str.contains() method, and if found, the 'dog_stage' column was updated with the appropriate dog_stage description.
-  The 'dog_stage' column in the twitter_archive_master dataframe was updated with any newly found dog_stage descriptions.

**4) 'twitter_archive_master" Table: Missing & Incorrect Entries in the 'name' Column**

-  A subset dataframe was created for the 'None', 'a', 'an' and 'the' entries in the 'name' column.
-  The 'text' column was queried for text related to tweets that were not based on a dog rating and these tweets were removed.
-  The 'text' column was queried for the correct dog names.  It was identified that most of the dog names followed the phrase 'named ...' or 'name is ...'. Therefore, the dog names were extracted from the text and the 'name' column was updated.


**5) 'image_prediction_clean' Table: Prediction Confidence Values as Percentages**

-  The prediction confidence values were changed to percentage value, by multiplying values by 100, as easier to understand during analysis, and the percentage values were converted from floats to ints.

**6) 'image_prediction_clean' Table: Column Headings**

-  The column headings of img_num, p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog were renamed to make them easier to understand: 'image_numenr', 'prediction_1', 'prediction_1_confidence(%)', 'prediction_1_dog' etc. 
